In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed: timeout during initial read of root folder; for more info: https://research.google.com/colaboratory/faq.html#drive-timeout

In [ ]:
import os
!pip install mne
import mne
import numpy as np
import pandas as pd
import math
import csv

from mne.io import read_raw_edf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 44.2 MB/s eta 0:00:00


In [ ]:
timeList=[]
timeLineList=[]
isHaveSeizure = True
generalAVG = 0
point = 0


def adjustmentStartEndTimeData(timeLineList):
        # Belirtilen satırdan sonraki sayıları al
        numbers = []
        time = ""
        for list in timeLineList:
            for satir in list[13:]:
                # Satırdaki sayıları bulma
                for word in satir.split():
                    try:
                        number = int(word)
                        numbers.append(number)
                    except ValueError:
                        pass  # Sayıya dönüştürülemezse geç
            for num in numbers:
                time+=str(num)
            timeList.append(time)
            numbers.clear()
            time = ""

def findStartEndTime(file_name, sentence):
    global isHaveSeizure
    isHaveSeizure = True
    timeLineList.clear()
    timeList.clear()
    with open(file_name, 'r', encoding='utf-8') as file:
        line_number = 0
        for line in file:
            line_number += 1
            if sentence in line:
                fileLines = file.readlines()
                # Summary dosyasından nöbet(seizure) start time ve end time bilgilerinin alınması
                if (file_name.endswith("24-summary.txt")):
                  startTimeLine = fileLines[1] # fileLines'da 4. satır start time'a denk geliyor
                  endTimeLine = fileLines[2] # fileLines'da 5. satır end time'a denk geliyor
                else:
                  startTimeLine = fileLines[3] # fileLines'da 3. satır start time'a denk geliyor
                  endTimeLine = fileLines[4] # fileLines'da 4. satır end time'a denk geliyor
                timeLineList.append(startTimeLine)
                timeLineList.append(endTimeLine)
                if not startTimeLine.startswith("Seizure"):
                    isHaveSeizure = False
                adjustmentStartEndTimeData(timeLineList)
                print("timelist: "+str(timeList))
                return f"Cümle bulundu: Satır {line_number}: {line}"
    return "Cümle bulunamadı."



In [ ]:
def fixColumns(file):
    readcsv = file
    try:
      readcsv = readcsv.drop(columns=["--0", "--1", "--2", "--3", "--4", "T8-P8-1"])
    except:
      pass

    try:
      readcsv = readcsv.drop(columns=[".-0", ".-1", ".-2", ".-3", ".-4", "T8-P8-1"])
    except:
      pass
    return readcsv.rename(columns={"T8-P8-0": "T8-P8", "Unnamed: 0":"num"})

In [ ]:
def get_column_names(csv_file):
    with open(csv_file, 'r', newline='') as file:
        reader = csv.DictReader(file)
        column_names = reader.fieldnames
    return column_names

In [ ]:
def addOutputColumn(df):
    try:
        df["outcome"] = 0.0
        df = fixColumns(df)
        if(isHaveSeizure):
            startTime = timeList[0]
            endTime = timeList[1]
            df.loc[(df['time'].astype(float) > float(startTime) - 1200) & (df['time'].astype(float) < float(endTime)), 'outcome'] = 0.5
            df.loc[(df['time'].astype(float) > float(startTime) - 300) & (df['time'].astype(float) < float(endTime)), 'outcome'] = 0.8
            df.loc[(df['time'].astype(float) > float(startTime)) & (df['time'].astype(float) < float(endTime)), 'outcome'] = 1.0
        df.to_csv("csv/"+output_file_name)
        postLabel(df)
    except FileNotFoundError:
        pass

In [ ]:
def filter(raw):
    low_cut = 50.0
    high_cut = 1.0
    raw_filt = raw.copy().filter(low_cut, high_cut)
    return raw_filt

In [ ]:
def checkAVG(partdf):
    global point
    point = 0
    columnstoavg = partdf.columns.difference(['time', 'outcome'])
    columnavg = partdf[columnstoavg].abs().mean()
    localAVG = columnavg.mean()
    localAVG *= 1.12
    if(generalAVG<localAVG):
        point+=1

In [ ]:
def check50(partdf):
    global point
    columns_to_check = partdf.columns.difference(['time', 'outcome'])
    partdf['abs_sum'] = partdf[columns_to_check].abs().sum(axis=1)
    partdf.loc[partdf['abs_sum'].div(23) > 50, 'exceeds_50'] = True
    true_count = partdf['exceeds_50'].sum()
    if(true_count>100):
      point+=1

In [ ]:
def checkSPO(partdf, encokgecen):
    global point
    if(encokgecen==0.8):
      point=10

In [ ]:
def checkLabel(partdf, encokgecen):
    global point
    checkAVG(partdf)
    check50(partdf)
    checkSPO(partdf, encokgecen)
    if(point>1):
        encokgecen = 0.5
    else:
        encokgecen = 0.0

    #checkFFT()

    # print(encokgecen)
    # print(generalAVG)
    # print(localAVG)

    return encokgecen

In [ ]:
from collections import Counter

def postLabel(df):
    global generalAVG
    global point
    filtereddf = df[(df['outcome'] != 1.0) & (df['outcome'] != 0.0)]
    columnstoavg = filtereddf.columns.difference(['time', 'outcome'])
    columnavg = filtereddf[columnstoavg].abs().mean()
    generalAVG = columnavg.mean()

    etiket_oncesi = []
    outcome_deger_list = []
    for i in range(0, len(df), 256):  # Step 1 saniye 256hZ yani
        aralik_df = df.iloc[i:i+1024]  # 4 Saniye aralığını seç
        #calculate_mean_abs(aralik_df)
        if(aralik_df.shape[0]==1024):
              degerler = aralik_df['outcome'].tolist()
              frekanslar = Counter(degerler)
              en_cok_gecen = frekanslar.most_common(1)[0][0]
              if en_cok_gecen not in [0.0, 1.0]:
                  en_cok_gecen = checkLabel(aralik_df, en_cok_gecen)
              etiket_oncesi.append(en_cok_gecen)
    label_name_list = []
    for i in range(1, len(etiket_oncesi)+1):
        labelname = os.path.splitext(edf_file)[0] + f'_{i}'
        label_name_list.append(labelname)

    etiketdf = pd.DataFrame({'part': label_name_list, 'label': etiket_oncesi})
    etiketdf.to_csv(f"/content/drive/MyDrive/Dataset_Models/dataset/trail/labelcsv8/{os.path.splitext(edf_file)[0]}_partlabel.csv")

In [ ]:
output_path = 'csv/'

file_list = os.listdir("/content/drive/MyDrive/Dataset_Models/dataset/trail/chb12")
edf_files = [file for file in file_list if file.endswith(".edf")]
data_path = '/content/drive/MyDrive/Dataset_Models/dataset/trail'
for edf_file in edf_files:
    #print(edf_file)
    subject_folder = edf_file[:5]  #işlem yapılan hasta
    print(subject_folder)

    all_files = [f for f in os.listdir(os.path.join(data_path, 'chb12')) if f.endswith('.edf')]
    sonuc = findStartEndTime(data_path +"/"+ 'chb12'+"/"+subject_folder+"-summary.txt", edf_file)
    #print(sonuc) # Start-End time print

    output_file_name = os.path.splitext(edf_file)[0] + f'_segment.csv'
    csv_file_path = os.path.join(output_path, output_file_name)

    subject_path = os.path.join(data_path, 'chb12')
    edf_file_path = os.path.join(subject_path, edf_file)

    # okue
    raw = read_raw_edf(edf_file_path, preload=True)
    raw = filter(raw)
    raw = raw.drop_channels(["--0", "--1", "--2", "--3", "--4", "T8-P8-1",
                                 "VNS", "ECG", "--5", "FC1-Ref", "FC2-Ref", "FC5-Ref", "FC6-Ref",
                                 "CP1-Ref", "CP2-Ref", "CP5-Ref", "CP6-Ref", ".-0", ".-1", ".-2",
                                 ".-3", ".-4", "T8-P8-1", "PZ-OZ", ".", "T8-P8-1"], on_missing='ignore')

    custom_order = ['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FZ-CZ', 'CZ-PZ', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8']  # Örneğin, özel bir sıralama
    raw.reorder_channels(custom_order)
    #########################################
    # print("\nTrimmed Data Info:")
    print(f"Number of channels: {len(raw.ch_names)}")
    print(f"Number of time points: {raw.times.shape[0]}")
    # print(f"Shape of the data array: {raw_trimmed.get_data().shape}")
    #########################################
    df = raw.to_data_frame()
    # Dosyanın adını ve parça numarasını kullanarak çıktı dosya adını oluştur

    file_list = os.listdir(output_path)

    addOutputColumn(df)
    print(csv_file_path)

chb11
timelist: ['1454', '2206']
Extracting EDF parameters from /content/drive/MyDrive/Dataset_Models/dataset/trail/chb12/chb11_99.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 731903  =      0.000 ...  2858.996 secs...


<ipython-input-14-a8fd3d968e48>:22: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8', '-'}. Applying running numbers for duplicates.
  raw = read_raw_edf(edf_file_path, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 845 samples (3.301 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.5s


Number of channels: 22
Number of time points: 731904


...

<ipython-input-14-a8fd3d968e48>:22: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8', '-'}. Applying running numbers for duplicates.
  raw = read_raw_edf(edf_file_path, preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 845 samples (3.301 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.6s


Number of channels: 22
Number of time points: 921600


KeyboardInterrupt: 

In [ ]:
dosya_listesi = os.listdir('/content/drive/MyDrive/Dataset_Models/dataset/trail/labelcsv8')
print(dosya_listesi)
df_listesi = []
for dosya in dosya_listesi:
    if dosya.endswith('partlabel.csv'):

        dosya_yolu = os.path.join('/content/drive/MyDrive/Dataset_Models/dataset/trail/labelcsv8', dosya)
        df = pd.read_csv(dosya_yolu)
        df_listesi.append(df)
        print(f"{dosya} dosyası okundu:")


birlesik_df = pd.concat(df_listesi)
print(birlesik_df['label'].value_counts())

['chb11_99_partlabel.csv', 'chb11_92_partlabel.csv', 'chb11_82_partlabel.csv', 'chb03_35_partlabel.csv', 'chb05_06_partlabel.csv', 'chb08_21_partlabel.csv', 'chb03_01_partlabel.csv', 'chb03_02_partlabel.csv', 'chb03_03_partlabel.csv', 'chb03_04_partlabel.csv', 'chb03_36_partlabel.csv', 'chb08_13_partlabel.csv', 'chb08_11_partlabel.csv', 'chb08_02_partlabel.csv', 'chb18_36_partlabel.csv', 'chb18_35_partlabel.csv', 'chb18_32_partlabel.csv', 'chb18_31_partlabel.csv', 'chb18_30_partlabel.csv', 'chb18_29_partlabel.csv', 'chb19_30_partlabel.csv', 'chb19_29_partlabel.csv', 'chb19_28_partlabel.csv', 'chb21_21_partlabel.csv', 'chb21_19_partlabel.csv', 'chb21_20_partlabel.csv', 'chb22_38_partlabel.csv', 'chb22_25_partlabel.csv', 'chb22_20_partlabel.csv', 'chb17_03_partlabel.csv', 'chb17_04_partlabel.csv', 'chb17_63_partlabel.csv', 'chb24_21_partlabel.csv', 'chb24_17_partlabel.csv', 'chb24_15_partlabel.csv', 'chb24_14_partlabel.csv', 'chb24_13_partlabel.csv', 'chb24_11_partlabel.csv', 'chb24_09_p

In [ ]:
from sklearn.utils import shuffle

df_interictal = birlesik_df[birlesik_df['label'] == 0.0]
df_preictal = birlesik_df[birlesik_df['label'] == 0.5]
df_ictal = birlesik_df[birlesik_df['label'] == 1.0]

df_interictal = shuffle(df_interictal)
df_interictal = df_interictal[0:7500]

df_preictal = shuffle(df_preictal)
df_preictal = df_preictal[0:7500]

df_ictal = shuffle(df_ictal)
df_ictal = df_ictal[0:7431]

df_data = pd.concat([df_interictal, df_preictal, df_ictal], axis=0)
df_data = shuffle(df_data)

print(df_data)

# UNUTMA: SADECE CHB12 KLASÖRÜNDEKİ VERİLER ALINIYOR

      Unnamed: 0           part  label
560          560   chb08_11_561    0.0
1470        1470  chb09_19_1471    0.0
864          864   chb15_22_865    1.0
3336        3336  chb15_62_3337    0.0
1610        1610  chb08_21_1611    0.5
...          ...            ...    ...
2854        2854  chb17_63_2855    0.5
2584        2584  chb17_04_2585    0.5
6784        6784  chb10_12_6785    0.0
839          839   chb15_52_840    1.0
1276        1276  chb10_31_1277    0.0

[22431 rows x 3 columns]


In [ ]:
df_data.to_csv("/content/drive/MyDrive/Dataset_Models/dataset/trail/labelcsv8/"+"traillabelreduced.csv")